### Importing libraries

In [237]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import functions
from pprint import pprint
%matplotlib inline

#### Importing Dataset

In [238]:
xls = pd.ExcelFile('2019 County Health Rankings Data - v2.xls')
df = pd.read_excel(xls, 'Additional Measure Data',skiprows=1)

In [239]:
df.head()

,FIPS,State,County,Life Expectancy,95% CI - Low,95% CI - High,Life Expectancy (Black),Life Expectancy (Hispanic),Life Expectancy (White),# Deaths,...,% Hispanic,# Non-Hispanic White,% Non-Hispanic White,# Not Proficient in English,% Not Proficient in English,95% CI - Low.16,95% CI - High.16,% Female,# Rural,% Rural
0,1001,Alabama,Autauga,76.330589,75.554117,77.107061,74.600628,NaN,76.337219,815.0,...,2.857452,41336,74.473912,430,0.828548,0.422472,1.234624,51.342246,22921.0,42.002162
1,1003,Alabama,Baldwin,78.599498,78.208357,78.990639,75.628919,98.039893,78.537003,2827.0,...,4.550200,176582,83.047388,872,0.454512,0.282144,0.626880,51.452772,77060.0,42.279099
2,1005,Alabama,Barbour,75.779457,74.639355,76.919559,74.240228,NaN,77.568384,451.0,...,4.206569,11613,45.955679,297,1.199273,0.580536,1.818011,47.229917,18613.0,67.789635
3,1007,Alabama,Bibb,73.928271,72.624611,75.231931,72.294686,NaN,74.109510,445.0,...,2.638080,16842,74.298571,84,0.394348,0.000000,0.933361,46.453150,15663.0,68.352607
4,1009,Alabama,Blount,74.597767,73.810999,75.384535,NaN,78.577094,74.098247,1050.0,...,9.565097,50439,86.944306,1013,1.870004,1.364180,2.375829,50.688639,51562.0,89.951502


In [240]:
df.select_dtypes(include=['object'])[:2]

,State,County,Other PCP Ratio
0,Alabama,Autauga,3265:1
1,Alabama,Baldwin,1916:1


### Exploratory Data Analysis

#### Renaming columns

In [241]:
columns_name = df.columns
new_column_names = []
for column_name in columns_name:
    new_column_names.append(column_name.replace(" ", "_"))
pprint(new_column_names[:5])
df.columns = new_column_names

['FIPS', 'State', 'County', 'Life_Expectancy', '95%_CI_-_Low']


#### Renaming counties

In [242]:
county_names = df["County"]
new_county_names = []
for county_name in county_names:
    new_county_names.append(county_name.replace(" ", "_"))
df["County"] = new_county_names

#### Dropping ethnicies columns and columns with a lot of missing values

In [243]:
to_drop = ["Other_PCP_Ratio", "FIPS", "%_Free_or_Reduced_Lunch", "#_HIV_Cases", "HIV_Prevalence_Rate"]
#Data is missing for HIV only in South Dakota, replace it by country weighted average later or look at this column in depth
containing = ["CI_-_Low", "CI_-_High", "White", "Hispanic", "Black", "Asian" ,"Alaskan", "Indian", "Hawaiian", "African"]
for column in df.columns:
    if (column in to_drop) or any(word in column for word in containing):
        df = df.drop([column], axis=1)

In [244]:
df.columns

Index(['State', 'County', 'Life_Expectancy', '#_Deaths',
       'Age-Adjusted_Mortality', '#_Deaths.1', 'Child_Mortality_Rate',
       '#_Deaths.2', 'Infant_Mortality_Rate', '%_Frequent_Physical_Distress',
       '%_Frequent_Mental_Distress', '%_Diabetic', '#_Food_Insecure',
       '%_Food_Insecure', '#_Limited_Access', '%_Limited_Access',
       '#_Drug_Overdose_Deaths', 'Drug_Overdose_Mortality_Rate',
       '#_Motor_Vehicle_Deaths', 'MV_Mortality_Rate', '%_Insufficient_Sleep',
       '#_Uninsured', '%_Uninsured', '#_Uninsured.1', '%_Uninsured.1',
       'Other_PCP_Rate', '%_Disconnected_Youth', 'Household_Income',
       'Segregation_index', 'Segregation_Index', 'Homicide_Rate',
       '#_Firearm_Fatalities', 'Firearm_Fatalities_Rate', '#_Homeowners',
       '%_Homeowners', '#_Households_with_Severe_Cost_Burden',
       '%_Severe_Housing_Cost_Burden', 'Population', '%_<_18', '%_65_and_over',
       '#_Not_Proficient_in_English', '%_Not_Proficient_in_English',
       '%_Female', '#_R

#### Delete rows missing Life Expectancy

In [245]:
df = df.dropna(subset=["Life_Expectancy"])

### Fill counties missing values using states weighted averages

In [246]:
df.head(2)

,State,County,Life_Expectancy,#_Deaths,Age-Adjusted_Mortality,#_Deaths.1,Child_Mortality_Rate,#_Deaths.2,Infant_Mortality_Rate,%_Frequent_Physical_Distress,...,#_Households_with_Severe_Cost_Burden,%_Severe_Housing_Cost_Burden,Population,%_<_18,%_65_and_over,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Rural,%_Rural
0,Alabama,Autauga,76.330589,815.0,438.7,29.0,53.279441,34.0,7.699275,12.824434,...,2569.0,12.645206,55504,23.937014,15.119631,430,0.828548,51.342246,22921.0,42.002162
1,Alabama,Baldwin,78.599498,2827.0,348.4,85.0,47.000017,91.0,5.851337,12.622002,...,9313.0,12.795922,212628,21.848487,19.947044,872,0.454512,51.452772,77060.0,42.279099


#### Calculate the weighted averages for each columns in each states

In [247]:
states = list(df.State.unique())
columns = [x for x in list(df.columns) if x not in ["State", "County"]]

In [248]:
state_wv = {}
for state in states:
    df_state = df.loc[df["State"] == state]
    for column in columns:
        values = df_state.dropna(subset=[column])[column]
        population_value = df_state.dropna(subset=[column])["Population"].astype(np.float64)
        population = np.sum(df_state.dropna(subset=[column])["Population"])
        if population != 0:
            wv = (values * population_value).sum() / population
            if not state_wv.get(state):
                state_wv[state] = {}
            state_wv[state][column] = float(wv)
        else:
            print(f"No data for {column} in {state}")

#### Fill each missing values by states weighted averages

In [253]:
indexes = list(df.index)
columns = list(df.columns)
df_filled = df.copy()
for index in indexes:
    for column in columns:
        if column not in ["State","County"]:
            if np.isnan(df[column][index]):
                df_filled[column][index] = state_wv[df["State"][index]][column]

/home/locsta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [254]:
df_filled.head()

,State,County,Life_Expectancy,#_Deaths,Age-Adjusted_Mortality,#_Deaths.1,Child_Mortality_Rate,#_Deaths.2,Infant_Mortality_Rate,%_Frequent_Physical_Distress,...,#_Households_with_Severe_Cost_Burden,%_Severe_Housing_Cost_Burden,Population,%_<_18,%_65_and_over,#_Not_Proficient_in_English,%_Not_Proficient_in_English,%_Female,#_Rural,%_Rural
0,Alabama,Autauga,76.330589,815.0,438.7,29.0,53.279441,34.0000,7.699275,12.824434,...,2569.0,12.645206,55504,23.937014,15.119631,430,0.828548,51.342246,22921.0,42.002162
1,Alabama,Baldwin,78.599498,2827.0,348.4,85.0,47.000017,91.0000,5.851337,12.622002,...,9313.0,12.795922,212628,21.848487,19.947044,872,0.454512,51.452772,77060.0,42.279099
2,Alabama,Barbour,75.779457,451.0,469.8,17.0,77.080027,204.8478,8.444680,16.216160,...,1158.0,13.586765,25270,20.763751,18.824693,297,1.199273,47.229917,18613.0,67.789635
3,Alabama,Bibb,73.928271,445.0,563.7,21.0,111.749681,25.0000,14.792899,13.162968,...,576.0,8.733889,22668,20.606141,16.022587,84,0.394348,46.453150,15663.0,68.352607
4,Alabama,Blount,74.597767,1050.0,501.8,41.0,76.032935,29.0000,6.184688,13.720057,...,1569.0,8.194495,58013,23.349939,17.842553,1013,1.870004,50.688639,51562.0,89.951502


In [255]:
df_filled.to_csv(r"/home/locsta/Documents/FlatIron/County-Health-Analysis/clean.csv")

#### Drop County and turn State column into category

In [ ]:
df_filled["State"] = df_filled["State"].astype('category')
df_filled = df_filled.drop(["County"], axis=1)

TODO: Transform every # into percentages

TODO: Rename Deaths

TODO: limited access = limited access to healthy foods

In [ ]:
"#_Deaths" = "Premature age-adjusted mortality"
"#_Deaths.1" = "Child mortality"
"#_Deaths.2" = "Infant mortality"